In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import sklearn
import xgboost as xgb
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import pickle

# read data

In [2]:
file = pd.read_csv('/Users/yanlixu/Desktop/git_code/machine_learning_prediction/molnet_bace.csv')
dataset = file[['mol', 'pIC50']]
train_data_x, test_data_x, train_y, test_y = train_test_split(dataset['mol'], dataset['pIC50'], test_size = 0.2, random_state = 1 )


In [3]:
len(train_data_x), len(test_data_x)

(1210, 303)

# calculate fingerPrint

In [4]:
train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x] # RDKit Mol object
train_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 4, 2048) for mol in train_mols]
train_x = np.asarray(train_fps, dtype = float)

test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x] # RDKit Mol object
test_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 4, 2048) for mol in test_mols]
test_x = np.asarray(test_fps, dtype = float)

# Grid search

In [5]:
# 1.参数字典
xgb_param_grid = {'n_estimators':[10, 50, 100],
                  'max_depth':[3, 6, 7, 8, 10]}


svm_param_dict = {'C':[1, 2, 3, 4, 5],
                  'kernel':['poly', 'rbf', 'sigmoid'],
                  'epsilon':[0.1, 0.5, 1.0]}

rf_param_dict = {'n_estimators':[50, 70, 100, 150, 200],
                 'max_depth':[10, 50, 100],
                 'max_features': ["auto","sqrt","log2"]}

knn_param_dict = {'n_neighbors':[5, 10, 15, 20],
                  'weights':['uniform', 'distance']}

# 2.性能指标字典
score_dict = {'mse':make_scorer(mean_squared_error),
              'mae':make_scorer(mean_absolute_error),
              'mape':make_scorer(mean_absolute_percentage_error),
              'r2':make_scorer(r2_score)}

In [6]:
# xgb model
xgb_reg = xgb.XGBRegressor()
xgb_gs = GridSearchCV(xgb_reg,
                      xgb_param_grid,
                      scoring = score_dict,
                      cv = 5,
                      refit = 'r2',
                      return_train_score = True)

# svm model
svm_reg = svm.SVR()
svm_gs = GridSearchCV(estimator = svm_reg,
                      param_grid = svm_param_dict,
                      scoring = score_dict,
                      n_jobs = 10,
                      cv = 10,
                      refit = 'r2',
                      return_train_score = True)
# random forest model
rf_reg = RandomForestRegressor()
rf_gs = GridSearchCV(estimator = rf_reg,
                     param_grid = rf_param_dict,
                     scoring = score_dict,
                     n_jobs = 10,
                     cv = 5, 
                     refit = 'r2',
                     return_train_score = True)

# KNN
knn_reg = KNeighborsRegressor()
knn_gs = GridSearchCV(estimator = knn_reg,
                      param_grid = knn_param_dict,
                      scoring = score_dict,
                      n_jobs = 10,
                      cv = 10,
                      refit = 'r2',
                      return_train_score = True)

In [7]:
# xgb
xgb_gs_ecfp = xgb_gs.fit(train_x, train_y)
xgb_model = xgb_gs_ecfp.best_estimator_
# svm
svm_gs_fit = svm_gs.fit(train_x, train_y)
svm_model = svm_gs_fit.best_estimator_


In [8]:
rf_gs_fit = rf_gs.fit(train_x, train_y)
rf_best_model = rf_gs_fit.best_estimator_

In [9]:
knn_gs_fit = knn_gs.fit(train_x, train_y)
knn_model = knn_gs_fit.best_estimator_

In [10]:
xgb_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=50, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [11]:
svm_model

SVR(C=3)

In [12]:
rf_best_model

RandomForestRegressor(max_depth=100, max_features='sqrt', n_estimators=200)

In [13]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

# Cross Validation of XGBoost, SVM

In [14]:
xgb_cv = cross_validate(xgb_model,
                        train_x,
                        train_y,
                        cv = 5,
                        n_jobs = 10,
                        scoring = score_dict,
                        return_train_score = True)

xgb_cv_train_mae = np.mean(xgb_cv['train_mae'])
xgb_cv_train_mse = np.mean(xgb_cv['train_mse'])
xgb_cv_train_mape = np.mean(xgb_cv['train_mape'])
xgb_cv_train_r2 = np.mean(xgb_cv['train_r2'])

xgb_cv_test_mae = np.mean(xgb_cv['test_mae'])
xgb_cv_test_mse = np.mean(xgb_cv['test_mse'])
xgb_cv_test_mape = np.mean(xgb_cv['test_mape'])
xgb_cv_test_r2 = np.mean(xgb_cv['test_r2'])

In [15]:
svm_best_cv = cross_validate(estimator = svm_model,
                             X = train_x,
                             y = train_y,
                             scoring = score_dict,
                             cv = 5,
                             n_jobs = 10,
                             return_train_score = True)

svm_cv_train_mae = np.mean(svm_best_cv['train_mae'])
svm_cv_train_mse = np.mean(svm_best_cv['train_mse'])
svm_cv_train_mape = np.mean(svm_best_cv['train_mape'])
svm_cv_train_r2 = np.mean(svm_best_cv['train_r2'])

svm_cv_test_mae = np.mean(svm_best_cv['test_mae'])
svm_cv_test_mse = np.mean(svm_best_cv['test_mse'])
svm_cv_test_mape = np.mean(svm_best_cv['test_mape'])
svm_cv_test_r2 = np.mean(svm_best_cv['test_r2'])

In [16]:
rf_best_cv = cross_validate(estimator = rf_best_model,
                            X = train_x,
                            y = train_y,
                            scoring = score_dict,
                            cv = 5,
                            n_jobs = 10,
                            return_train_score = True)

rf_cv_train_mae = np.mean(rf_best_cv['train_mae'])
rf_cv_train_mse = np.mean(rf_best_cv['train_mse'])
rf_cv_train_mape = np.mean(rf_best_cv['train_mape'])
rf_cv_train_r2 = np.mean(rf_best_cv['train_r2'])

rf_cv_test_mae = np.mean(rf_best_cv['test_mae'])
rf_cv_test_mse = np.mean(rf_best_cv['test_mse'])
rf_cv_test_mape = np.mean(rf_best_cv['test_mape'])
rf_cv_test_r2 = np.mean(rf_best_cv['test_r2'])

In [17]:
knn_best_cv = cross_validate(estimator = knn_model,
                             X = train_x,
                             y = train_y,
                             scoring = score_dict,
                             cv = 5,
                             n_jobs = 10,
                             return_train_score = True)

knn_cv_train_mae = np.mean(knn_best_cv['train_mae'])
knn_cv_train_mse = np.mean(knn_best_cv['train_mse'])
knn_cv_train_mape = np.mean(knn_best_cv['train_mape'])
knn_cv_train_r2 = np.mean(knn_best_cv['train_r2'])

knn_cv_test_mae = np.mean(knn_best_cv['test_mae'])
knn_cv_test_mse = np.mean(knn_best_cv['test_mse'])
knn_cv_test_mape = np.mean(knn_best_cv['test_mape'])
knn_cv_test_r2 = np.mean(knn_best_cv['test_r2'])

# external test

In [18]:
xgb_ext_pred = xgb_model.predict(test_x)
svm_ext_pred = svm_model.predict(test_x)
rf_ext_pred = rf_best_model.predict(test_x)
knn_ext_pred = knn_model.predict(test_x)

ext_test_df =  pd.DataFrame({'mol':test_data_x.tolist(),
                            'true_pIC50':test_y.tolist(),
                            'xgb_test_pIC50':xgb_ext_pred,
                            'svm_test_pIC50':svm_ext_pred,
                            'rf_test_pIC50':rf_ext_pred,
                            'knn_test_pIC50':knn_ext_pred})
ext_test_df

,mol,true_pIC50,xgb_test_pIC50,svm_test_pIC50,rf_test_pIC50,knn_test_pIC50
0,S1(=O)(=O)N(CCCC1)c1cc(cc(c1)/C(=N\OCc1ccccc1)...,7.508638,7.363010,7.198399,7.341457,7.125901
1,Clc1cc2CC(N=C(NC(Cc3cscc3-c3cn[nH]c3)C(=O)[O-]...,6.346787,6.471633,6.356097,6.227854,6.060866
2,Clc1cc2CC(N=C(NC(Cc3cscc3CCC3CC3)C(=O)[O-])c2c...,6.244125,6.008316,6.222500,6.090137,6.267117
3,FC(F)(F)c1cc(ccc1)C[NH2+]CC(O)C(NC(=O)c1cc(N2C...,7.552842,7.298934,7.553792,7.484408,7.417836
4,Fc1cc(cc(F)c1)CC(NC(=O)C)C(O)C[NH2+]C1(CCCCC1)...,5.801343,4.711882,5.575482,5.647281,4.777851
...,...,...,...,...,...,...
298,OC(C(NC(=O)c1cc(N(C(=O)C)c2ccccc2)ccc1)Cc1cccc...,5.886056,6.891860,6.404833,6.616499,6.361417
299,S(=O)(=O)(N(Cc1ccccc1)c1cc(ccc1)C(=O)NC(Cc1ccc...,6.744728,7.374022,6.404952,6.722379,6.870230
300,FCCCCC#Cc1cc(ccc1)[C@]1(N=C(N)N(C)C1=O)c1ccc(O...,7.397940,7.690667,7.750289,7.739008,7.844370
301,Clc1cc2nc(n(c2cc1)C(CC(C)C)CC(=O)NC(C)C)N,3.609065,3.812004,3.734805,3.996534,3.848633


In [19]:
ext_test_df.to_csv('molnet_bace_regression_models_prediction_results.csv')

# final results summary

In [26]:
xgb_ext_mae = mean_absolute_error(test_y, xgb_ext_pred)
xgb_ext_mse = mean_squared_error(test_y, xgb_ext_pred)
xgb_ext_mape = mean_absolute_percentage_error(test_y, xgb_ext_pred)
xgb_ext_r2 = r2_score(test_y, xgb_ext_pred)

xgb_perf = {'mae':[xgb_cv_train_mae, xgb_cv_test_mae, xgb_ext_mae],
            'mse':[xgb_cv_train_mse, xgb_cv_test_mse, xgb_ext_mse],
            'mape':[xgb_cv_train_mape, xgb_cv_test_mape, xgb_ext_mape],
            'r2':[xgb_cv_train_r2, xgb_cv_test_r2, xgb_ext_r2]}

xgb_perf_df = pd.DataFrame.from_dict(xgb_perf)
xgb_perf_df.index = ['train','cv', 'ext']
round(xgb_perf_df, 2)

,mae,mse,mape,r2
train,0.19,0.07,0.03,0.96
cv,0.59,0.62,0.10,0.66
ext,0.56,0.59,0.09,0.65


In [27]:
svm_ext_mae = mean_absolute_error(test_y, svm_ext_pred)
svm_ext_mse = mean_squared_error(test_y, svm_ext_pred)
svm_ext_mape = mean_absolute_percentage_error(test_y, svm_ext_pred)
svm_ext_r2 = r2_score(test_y, svm_ext_pred)

svm_perf = {'mae':[svm_cv_train_mae, svm_cv_test_mae, svm_ext_mae],
            'mse':[svm_cv_train_mse, svm_cv_test_mse, svm_ext_mse],
            'mape':[svm_cv_train_mape, svm_cv_test_mape, svm_ext_mape],
            'r2':[svm_cv_train_r2, svm_cv_test_r2, svm_ext_r2]}

svm_perf_df = pd.DataFrame.from_dict(svm_perf)
svm_perf_df.index = ['train','cv', 'ext']
round(svm_perf_df, 2)

,mae,mse,mape,r2
train,0.12,0.04,0.02,0.98
cv,0.52,0.49,0.09,0.73
ext,0.48,0.44,0.08,0.74


In [28]:
rf_ext_mae = mean_absolute_error(test_y, rf_ext_pred)
rf_ext_mse = mean_squared_error(test_y, rf_ext_pred)
rf_ext_mape = mean_absolute_percentage_error(test_y, rf_ext_pred)
rf_ext_r2 = r2_score(test_y, rf_ext_pred)

rf_perf = {'mae':[rf_cv_train_mae, rf_cv_test_mae, rf_ext_mae],
            'mse':[rf_cv_train_mse, rf_cv_test_mse, rf_ext_mse],
            'mape':[rf_cv_train_mape, rf_cv_test_mape, rf_ext_mape],
            'r2':[rf_cv_train_r2, rf_cv_test_r2, rf_ext_r2]}

rf_perf_df = pd.DataFrame.from_dict(rf_perf)
rf_perf_df.index = ['train','cv', 'ext']
round(rf_perf_df, 2)

,mae,mse,mape,r2
train,0.22,0.09,0.04,0.95
cv,0.58,0.60,0.10,0.67
ext,0.55,0.57,0.09,0.66


In [29]:
knn_ext_mae = mean_absolute_error(test_y, knn_ext_pred)
knn_ext_mse = mean_squared_error(test_y, knn_ext_pred)
knn_ext_mape = mean_absolute_percentage_error(test_y, knn_ext_pred)
knn_ext_r2 = r2_score(test_y, knn_ext_pred)

knn_perf = {'mae':[knn_cv_train_mae, knn_cv_test_mae, knn_ext_mae],
            'mse':[knn_cv_train_mse, knn_cv_test_mse, knn_ext_mse],
            'mape':[knn_cv_train_mape, knn_cv_test_mape, knn_ext_mape],
            'r2':[knn_cv_train_r2, knn_cv_test_r2, knn_ext_r2]}

knn_perf_df = pd.DataFrame.from_dict(knn_perf)
knn_perf_df.index = ['train','cv', 'ext']
round(knn_perf_df, 2)

,mae,mse,mape,r2
train,0.48,0.45,0.08,0.75
cv,0.60,0.72,0.10,0.60
ext,0.57,0.71,0.09,0.58


In [24]:
regression_res = pd.concat([xgb_perf_df, svm_perf_df, rf_perf_df, knn_perf_df], keys=['xgb', 'svm', 'rf', 'knn'])
regression_res.to_csv('molnet_bace_xgb_svm_rf_knn_regression_metrics.csv')

# save model

In [25]:
with open('/Users/yanlixu/Desktop/git_code/machine_learning_prediction/xgb_reg_molnet_bace.pkl', 'wb') as file:
    pickle.dump(xgb_model, file)
with open('/Users/yanlixu/Desktop/git_code/machine_learning_prediction/svm_reg_molnet_bace.pkl', 'wb') as file:
    pickle.dump(svm_model, file)
with open('/Users/yanlixu/Desktop/git_code/machine_learning_prediction/rf_reg_molnet_bace.pkl', 'wb') as file:
    pickle.dump(rf_best_model, file)
with open('/Users/yanlixu/Desktop/git_code/machine_learning_prediction/knn_reg_molnet_bace.pkl', 'wb') as file:
    pickle.dump(knn_model, file)